# Pre-processing ISOX files from orbitrap


# Minimum

In [ ]:
def preproc_isox(path, df1_split_times,df2_split_times,df3_split_times,df4_split_times):
    import pandas as pd
    import os
    #read in the file from the specified path
    print(type(path))
    
    data_isox = pd.read_table(path, sep='\s+')
    
    #print info
    print('The total length of the file in minutes is from \n' + str(data_isox['time.min'].min())+' min to '+
      str(data_isox['time.min'].max()) + ' min')
    print('____________________________________')
    print('The chosed split of the file is into 4 files \n 1st ' 
        + str(df1_split_times) +
        ' min \n 2nd ' + str(df2_split_times) +
        ' min \n 3rd ' + str(df3_split_times) +
        ' min \n 4th ' + str(df4_split_times) +' min')

    #split the dataframe into 4 based on provided data
    df1 = data_isox[(data_isox['time.min'] > df1_split_times[0]) & (data_isox['time.min'] < df1_split_times[1])] 
    df2 = data_isox[(data_isox['time.min'] > df2_split_times[0]) & (data_isox['time.min'] < df2_split_times[1])]
    df3 = data_isox[(data_isox['time.min'] > df3_split_times[0]) & (data_isox['time.min'] < df3_split_times[1])]
    df4 = data_isox[(data_isox['time.min'] > df4_split_times[0]) & (data_isox['time.min'] < df4_split_times[1])]
    
    # reset times and scan number in the new dataframes
    dfs_list=[df1,df2,df3,df4]
    for df in dfs_list:
        df.loc[:,'time.min']=df['time.min']-df['time.min'].min()+0.01 
        df.loc[:,'scan.no']=df['scan.no']-df['scan.no'].min()+1

    # #write down the dataframes
    print('____________________________________')
    print('Current file is ' + path)
    print('Currently saving:')
    
    for df in dfs_list:
        outpath=df['filename'].iloc[0]
        os.makedirs(outpath, exist_ok=True)  
        name=(outpath +'/'+ df['filename'].iloc[0]+'_'+str(df['compound'].min())+'_'+str(df['compound'].max())+'.isox')
        
        print(name)
        df.to_csv(name, sep='\t',float_format='%.3f')
    
    print('New files are saved in ' + outpath)

In [ ]:
def main():
    import glob
    import pandas as pd
    import os
    
    paths=glob.glob(os.path.join('../data/',"*.isox"))
    #path=glob.glob(os.path.join("./*.isox"))
    # Set path where to save new files
    #outpath='../data/'

    # Set split times for the new files
    df1_split_times=[0,120]
    df2_split_times=[120.05,210]
    df3_split_times=[210.05,300]
    df4_split_times=[300.05,330]

    # this lines runs the function
    for path in paths:
        # do not process if folder exists 
        #print('path '+ path)
        #print((path.split('/')[2]).split('.')[0])
        fold_name=path.split('.')[0]
        isExist=os.path.exists('./'+fold_name)
        if not isExist:
            print('does not exist -> start processing')
            preproc_isox(path, df1_split_times,df2_split_times,df3_split_times,df4_split_times)
        else:
            print(fold_name +' already exists')
            continue
if __name__ == "__main__":
    main() 

In [ ]:
import glob
import os
#!ls
paths=glob.glob(os.path.join("*.isox"))
for path in paths:
    print(path)
    fold_name=path.split('.')[0]
    print(fold_name)

In [ ]:
(path.split('/')[2]).split('.')[0]

## missing compound string check

In [ ]:
#for every new file calculate percentage of 'lost' compounds


In [355]:
import pandas as pd
path='../data/52_Cholesterol-Nudibranch-57_125-400.isox'
data_isox = pd.read_table(path, sep='\s+')
#split times
df1_split_times = [0,120]
df2_split_times = [120.05,210]
df3_split_times = [210.05,300]
df4_split_times = [300.05,330]
#split the dataframe into 4 based on provided data
df1 = data_isox[(data_isox['time.min'] > df1_split_times[0]) & (data_isox['time.min'] < df1_split_times[1])] 
df2 = data_isox[(data_isox['time.min'] > df2_split_times[0]) & (data_isox['time.min'] < df2_split_times[1])]
df3 = data_isox[(data_isox['time.min'] > df3_split_times[0]) & (data_isox['time.min'] < df3_split_times[1])]
df4 = data_isox[(data_isox['time.min'] > df4_split_times[0]) & (data_isox['time.min'] < df4_split_times[1])]
# reset times and scan number in the new dataframes
dfs_list = [df1,df2,df3,df4]
for df in dfs_list:
    df.loc[:,'time.min'] = df['time.min']-df['time.min'].min()+0.01 
    df.loc[:,'scan.no'] = df['scan.no']-df['scan.no'].min()+1

In [367]:
df4

,filename,scan.no,time.min,compound,isotopolog,ions.incremental,tic,it.ms,intensity,resolution,...,peakNoise,mzMeasured,basePeakIntensity,rawOvFtT,intensCompFactor,agc,agcTarget,microscans,numberLockmassesFound,analyzerTemperature
548735,52_Cholesterol-Nudibranch-57_125-400,1,0.010,351,M0,2814.111945,32580944.0,40.271,1.659637e+05,35000,...,1035.976,351.26932,9925125.0,1226578.6,1.0725,0,5000000,5,0,44.17
548736,52_Cholesterol-Nudibranch-57_125-400,1,0.010,351,13C,844.891601,32580944.0,40.271,5.004923e+04,35000,...,1040.577,352.27234,9925125.0,1226578.6,1.0725,0,5000000,5,0,44.17
548737,52_Cholesterol-Nudibranch-57_125-400,1,0.010,353,M0,3485.845281,32580944.0,40.271,2.074145e+05,35000,...,1045.223,353.28500,9925125.0,1226578.6,1.0725,0,5000000,5,0,44.17
548738,52_Cholesterol-Nudibranch-57_125-400,1,0.010,353,13C,932.467950,32580944.0,40.271,5.572790e+04,35000,...,1049.824,354.28812,9925125.0,1226578.6,1.0725,0,5000000,5,0,44.17
548739,52_Cholesterol-Nudibranch-57_125-400,1,0.010,369,M0,158016.390271,32580944.0,40.271,9.925125e+06,35000,...,1103.346,369.35245,9925125.0,1226578.6,1.0725,0,5000000,5,0,44.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568662,52_Cholesterol-Nudibranch-57_125-400,2489,29.948,353,13C,768.774737,46803956.0,26.711,6.702911e+04,35000,...,1531.589,354.28833,14584427.0,1183167.1,1.0586,0,5000000,5,0,44.22
568663,52_Cholesterol-Nudibranch-57_125-400,2489,29.948,369,M0,159670.285537,46803956.0,26.711,1.458443e+07,35000,...,1604.513,369.35251,14584427.0,1183167.1,1.0586,0,5000000,5,0,44.22
568664,52_Cholesterol-Nudibranch-57_125-400,2489,29.948,369,13C,49201.472690,46803956.0,26.711,4.504249e+06,35000,...,1608.134,370.35565,14584427.0,1183167.1,1.0586,0,5000000,5,0,44.22
568665,52_Cholesterol-Nudibranch-57_125-400,2489,29.948,383,M0,135152.624271,46803956.0,26.711,1.273319e+07,35000,...,1654.972,383.33151,14584427.0,1183167.1,1.0586,0,5000000,5,0,44.22


In [ ]:
# try using shift method https://stackoverflow.com/questions/30673209/pandas-compare-next-row 
# df['qty_s'] = df['qty'].shift(-1)
# df['t_s'] = df['t'].shift(-1)
# df['z_s'] = df['z'].shift(-1)


df3.loc[:,'compound_s']= df3.loc[:,'compound'].shift(-1)

In [ ]:
df3.iloc[::2].head()

In [ ]:
df3['compound'].unique()


In [ ]:
#number of scans
df3['scan.no'].max()

In [ ]:
#number of isotopologs /2 (should be equal to number of scans)
df3[(df3['compound']==299)]['isotopolog'].count()/2

In [ ]:
compounds=df3['compound'].unique()
#print compound is its quantity does not match to number of scans
prob_comps=[comp for comp in compounds if df3[(df3['compound']==comp)]['isotopolog'].count()/2 != df3['scan.no'].max()]
print(prob_comps)
#[print(df3[(df3['compound']==prob_co)]['isotopolog'].count()) for prob_co in prob_comps]

In [ ]:
df3[(df3['compound']==301)]['isotopolog'].count()/2

In [ ]:
df2[df2['compound']==340]['isotopolog'].count()

In [ ]:
#info about the data

def percent(part, whole):
    try:
        return 100 * float(part) / float(whole)
    except ZeroDivisionError:
        return 0
    
for prob_co in prob_comps:
    numm_scans=(df3['scan.no'].max())
    num_comp=df3[df3['compound']==prob_co]['isotopolog'].count()/2

    print('compound: '+str(prob_co))
    print('number of scans: ' +str(numm_scans))
    print('number of compounds:')
    print(num_comp)
    #calculate percent of missing data
    perc=100 - percent(num_comp,numm_scans)
    print(f'missing {perc:.2f} % of the data' )
    print('___________')

In [ ]:
df3['isotopolog']=='13C'

In [ ]:
#find scans with missing data (number of isotopologues M0 and 13C is not equal)
# for  each_scan in scans:
#     if count(isotopolog=='13C') != count(isotopolog=='M0')
#     remove line
for prob_co in prob_comps:
    print('________ Compound: '+str(prob_co))
    for each_scan in range(1, df3['scan.no'].max()):
        sum13c=(df3[(df3['scan.no']==each_scan) & (df3['compound']==prob_co) & (df3['isotopolog']=='13C')]['compound'].count())
        #print(sum13c)
        sumM0=(df3[(df3['scan.no']==each_scan) & (df3['compound']==prob_co) & (df3['isotopolog']=='M0')]['compound'].count())
        if sum13c!=sumM0:
            print('Number of scan: ' +str(each_scan)+ '; there is ' + str(sum13c) +' 13C and ' + str(sumM0) + ' M0') 
        #df3.drop(sum13c!=sumM0)



In [ ]:
df3[(df3['scan.no']==each_scan) & (df3['compound']==prob_co) & (df3['isotopolog']=='13C')]

In [ ]:
# group by scan number 
pairs = df3.groupby(['scan.no','compound'])['isotopolog'].size().reset_index(name='counts')

# create lists to delete  
to_del_scanno = pairs[(pairs['counts'] != 2)]['scan.no'].values
to_del_compound = pairs[(pairs['counts'] !=2 )]['compound'].values

# keep only nesessary data in dataframe
df3_clean = df3[~(df3['scan.no'].isin(to_del_scanno) & df3['compound'].isin(to_del_compound))]
#df3_clean.info()

In [ ]:
dfs_list

In [380]:
#finalising
def percent(part, whole):
    try:
        return 100 * float(part) / float(whole)
    except ZeroDivisionError:
        return 0

dfs_list=[df1,df2,df3,df4]
i=1
def find_prob_comp(dfs_list):
    for df in dfs_list:
        print('___Working on file #'+str(i) )
        i=i+1
        #find problematic compunds
        compounds=df['compound'].unique()
        #print(compounds)
        #print compound if its quantity does not match to number of scans
        prob_comps=[comp for comp in compounds if df[(df['compound']==comp)]['isotopolog'].count()/2 != df['scan.no'].max()]
        #print(prob_comps)
        
        # for each problmatic compound print number of scans and number of compounds   
        for prob_co in prob_comps:
            numm_scans=(df['scan.no'].max())
            num_comp=df[df['compound']==prob_co]['isotopolog'].count()/2
            #calculate percent of missing data
            perc = 100 - percent(num_comp,numm_scans)
            print('Problematic compound: '+str(prob_co) +'; No of scans/compounds: ' +str(numm_scans)+' / '  +str(num_comp) )
            print(f'   missing {perc:.2f} % of the data' )

___Working on file #1
___Working on file #2
Problematic compound: 199; No of scans/compounds: 5539 / 5500.5
   missing 0.70 % of the data
Problematic compound: 261; No of scans/compounds: 5539 / 4220.5
   missing 23.80 % of the data
Problematic compound: 271; No of scans/compounds: 5539 / 5531.0
   missing 0.14 % of the data
Problematic compound: 272; No of scans/compounds: 5539 / 5538.5
   missing 0.01 % of the data
___Working on file #3
Problematic compound: 301; No of scans/compounds: 2563 / 2562.0
   missing 0.04 % of the data
Problematic compound: 313; No of scans/compounds: 2563 / 2560.5
   missing 0.10 % of the data
Problematic compound: 327; No of scans/compounds: 2563 / 2560.5
   missing 0.10 % of the data
Problematic compound: 340; No of scans/compounds: 2563 / 665.5
   missing 74.03 % of the data
___Working on file #4
Problematic compound: 383; No of scans/compounds: 2489 / 2488.0
   missing 0.04 % of the data
Problematic compound: 349; No of scans/compounds: 2489 / 11.0
   

In [394]:

def clean_df(dfs_list):
    dfs_list_new=[]
    for df in dfs_list:
        # group by scan number 
        pairs = df.groupby(['scan.no','compound'])['isotopolog'].size().reset_index(name='counts')

        # create lists to delete  
        to_del_scanno = pairs[(pairs['counts'] != 2)]['scan.no'].values
        to_del_compound = pairs[(pairs['counts'] !=2 )]['compound'].values

        # keep only nesessary data in dataframe
        df_clean = df[~(df['scan.no'].isin(to_del_scanno) & df['compound'].isin(to_del_compound))]
        dfs_list_new.append(df)
        #yield df
    return dfs_list_new
    # my_gen=clean_df(dfs_list)

    # print(my_gen)
    # for df in my_gen:
    #     dfs_l

In [395]:
dfs_list_new

[                                    filename  scan.no  time.min  compound   
 0       52_Cholesterol-Nudibranch-57_125-400        1     0.010       133  \
 1       52_Cholesterol-Nudibranch-57_125-400        1     0.010       133   
 2       52_Cholesterol-Nudibranch-57_125-400        1     0.010       135   
 3       52_Cholesterol-Nudibranch-57_125-400        1     0.010       135   
 4       52_Cholesterol-Nudibranch-57_125-400        1     0.010       145   
 ...                                      ...      ...       ...       ...   
 220875  52_Cholesterol-Nudibranch-57_125-400    10040   119.992       165   
 220876  52_Cholesterol-Nudibranch-57_125-400    10040   119.992       173   
 220877  52_Cholesterol-Nudibranch-57_125-400    10040   119.992       173   
 220878  52_Cholesterol-Nudibranch-57_125-400    10040   119.992       175   
 220879  52_Cholesterol-Nudibranch-57_125-400    10040   119.992       175   
 
        isotopolog  ions.incremental          tic   it.ms     

In [386]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19932 entries, 548735 to 568666
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   filename               19932 non-null  object 
 1   scan.no                19932 non-null  int64  
 2   time.min               19932 non-null  float64
 3   compound               19932 non-null  int64  
 4   isotopolog             19932 non-null  object 
 5   ions.incremental       19932 non-null  float64
 6   tic                    19932 non-null  float64
 7   it.ms                  19932 non-null  float64
 8   intensity              19932 non-null  float64
 9   resolution             19932 non-null  int64  
 10  peakResolution         19932 non-null  int64  
 11  peakNoise              19932 non-null  float64
 12  mzMeasured             19932 non-null  float64
 13  basePeakIntensity      19932 non-null  float64
 14  rawOvFtT               19932 non-null  float64
 15  i

In [ ]:
pairs

In [ ]:
pairs[(pairs['counts'] !=2 )]['compound'].values

In [ ]:
df3.info()

In [ ]:
df3_clean

In [ ]:
odd_df['compound']!=odd_df['compound_s']

In [ ]:
list([odd_df['compound']!=odd_df['compound_s'].index])

In [ ]:
(df3['compound'] == df3['compound_s']) 